<h1 style="text-align:center">Segmenting and Clustering Neighbourhood in Toronto</h1>

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
print("Library successfully imported")

Library successfully imported


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results=requests.get(url).text
soup = BeautifulSoup(results)


In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df=df.sort_values('PostalCode')
df.shape

(103, 3)

In [5]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
gta_longlat=pd.read_csv(url)
gta_longlat=gta_longlat.sort_values('Postal Code')
gta_longlat.shape

(103, 3)

The code below is use to merge the longitude and latitude value of gta_longlat to gta_df

In [6]:
gta_df = df
gta_df['Latitude'] = gta_longlat['Latitude']
gta_df['Longitude']= gta_longlat['Longitude']
gta_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
22,M1G,Scarborough,Woburn,43.770120,-79.408493
26,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
...,...,...,...,...,...
64,M9N,York,Weston,43.696948,-79.411307
70,M9P,Etobicoke,Westmount,43.648429,-79.382280
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.647927,-79.419750
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.602414,-79.543484


<h1>Clustering and map analysis</h1>

In [7]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="gta_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(gta_df['Latitude'], gta_df['Longitude'], gta_df['Borough'], gta_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3> Four Square</h3>

In [9]:
CLIENT_ID = 'S1FZEI0DWVJS2SOMSWK0ARJATDOT432HJBYZ0PIEXF0QTJUR' # your Foursquare ID
CLIENT_SECRET = 'FBMBEJ5ZKIK2DKDW2RT0LMCZCONF5BHNURYIPUBNHNIST4ZO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venue= getNearbyVenues(names=gta_df['Neighborhood'],
                               latitudes=gta_df['Latitude'],
                               longitudes=gta_df['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [12]:
toronto_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
1,"Malvern, Rouge",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
2,"Malvern, Rouge",43.727929,-79.262029,Dollarama,43.727092,-79.265784,Discount Store
3,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
4,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029,El Pulgarcito,43.792648,-79.259208,Latin American Restaurant


In [13]:
toronto_venue.shape

(2123, 7)

In [14]:
print('There are {} uniques categories.'.format(len(toronto_venue['Venue Category'].unique())))

There are 273 uniques categories.


<h3>Analysation of neighborhood</h3>

Generate dummies value of venue cateogry

In [15]:
toronto_onehot = pd.get_dummies(toronto_venue[['Venue Category']],prefix="",prefix_sep="")
toronto_onehot['Neighbourhood']=toronto_venue['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_onehot.shape

(2123, 274)

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.047619,0.000000,0.0,0.0,0.047619
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022472,0.0,...,0.0,0.0,0.011236,0.000000,0.000000,0.000000,0.011236,0.0,0.0,0.000000
4,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,...,0.0,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.026316
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.030303,0.000000,0.000000,0.0,0.0,0.000000
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
98,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.029412,0.000000,0.000000,0.000000,0.0,0.0,0.029412


In [18]:
toronto_grouped.shape

(100, 274)

The code below is use to find the top 10 venue for model evaluation purposes

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Nightclub,Burrito Place,Pet Store,Stadium,Furniture / Home Store,Climbing Gym
1,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Intersection,Shopping Mall,Park,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Fried Chicken Joint,Supermarket
2,Bayview Village,Indian Restaurant,Yoga Studio,Discount Store,Burger Joint,Sandwich Place,Restaurant,Coffee Shop,Pizza Place,Park,Fast Food Restaurant
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Restaurant,Cosmetics Shop,Bakery,Cocktail Bar,Clothing Store,Moroccan Restaurant,Department Store,Beer Bar
4,"Birch Cliff, Cliffside West",Coffee Shop,Clothing Store,Café,Restaurant,Thai Restaurant,Hotel,Gym,Bar,Cosmetics Shop,Sushi Restaurant


Prediction of cluster:

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 2, 3, 3], dtype=int32)

In [22]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = gta_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029,3.0,Discount Store,Department Store,Coffee Shop,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029,3.0,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Convenience Store,Japanese Restaurant,Shoe Store,Bank,Bakery,Juice Bar
22,M1G,Scarborough,Woburn,43.770120,-79.408493,3.0,Ramen Restaurant,Shopping Mall,Coffee Shop,Middle Eastern Restaurant,Café,Pizza Place,Restaurant,Juice Bar,Movie Theater,Sushi Restaurant
26,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,3.0,Japanese Restaurant,Baseball Field,Athletics & Sports,Gym,Café,Caribbean Restaurant,Modern European Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant


In [23]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].replace(np.nan,6)
toronto_merged['Cluster Labels'].unique()

array([3., 2., 4., 1., 0., 6.])

In [24]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int')

<h3>Visualization of clustering in Folium map</h3>

Since there is outlier in the prediction therefore i set the outlier to cluster 6 and print it is outlier in map

In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
rainbow = ['blue','green','red','black','yellow','gray']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if cluster==6:
        label = folium.Popup(str(poi) + ' Cluster ' + 'Outlier', parse_html=True)
    else:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters